In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-marathi/classification


In [7]:
path = Path('./')

In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-train.csv', header=None)
df_train.head()

,0,1
0,entertainment,…म्हणून सानिया मिर्झाची ड्यू डेट आहे खास
1,entertainment,Video : दीपिका-रणवीरच्या लग्नाचा मेन्यू झाला लीक!
2,sports,"सचिनच्या रणजी कारकीर्दीचा शेवट गोड, मुंबईचा विजय"
3,state,पुरंदरेंना महाराष्ट्र भूषण पुरस्काराविरोधात सं...
4,sports,"जयपूर पोलिसांच्या जाहिरातीवर बुमराह संतापला, स..."


In [9]:
Counter(list(df_train[0]))

Counter({'entertainment': 2660, 'sports': 984, 'state': 6028})

In [10]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-valid.csv', header=None)
df_valid.head()

,0,1
0,sports,"CWG 2018 : संजिता चानूची सुवर्णभरारी, भारताच्य..."
1,state,कर्मचार्‍यांच्या हलगर्जीपणामुळे होणार होते जिव...
2,sports,श्रीलंकेचा भारतावर 7 गडी राखून विजय
3,state,औरंगाबाद महापौरपदाचा तिढा अखेर सुटला
4,state,औरंगाबाद विद्यापीठाचा भोंगळ कारभार; गोदामात दि...


In [11]:
df_test = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/mr/mr-test.csv', header=None)
df_test.head()

,0,1
0,entertainment,काजोल पुन्हा माझ्या आयुष्यात येणार नाही - करण ...
1,entertainment,विराटला चीअर करण्यासाठी अनुष्का सिडनीत
2,entertainment,संतोषचा अड्डा!
3,state,मुरूड समुद्रात बुडालेल्या विद्यार्थ्याचा मृतदे...
4,state,"गोव्यात शिवसेना वेलिंगकरांसोबत,लवकरच युतीची घोषणा"


In [12]:
df_train.shape, df_valid.shape, df_test.shape

((9672, 2), (1210, 2), (1210, 2))

In [13]:
cutoff = int(0.05*len(df_train))

In [14]:
# from inltk.inltk import get_similar_sentences, tokenize
# import warnings
# warnings.filterwarnings("ignore")
# # aug_sen = []
# # aug_label = []
# for index, row in df_train[len(aug_sen)//2:cutoff].iterrows():
#     augs = get_similar_sentences(row[1], no_of_variations=2, language_code='mr', degree_of_aug=0.3)
#     aug_sen.extend(augs)
#     aug_label.extend([row[0]]*len(augs))
# df_aug = pd.DataFrame({0: aug_label, 1: aug_sen })
# df_aug.head()
# df_aug.to_csv('inltk_headlines_mr_train_aug.csv', index=False)

In [15]:
df_train_aug = pd.read_csv('inltk_headlines_mr_train_aug.csv')
df_train_aug.head()

,0,1
0,entertainment,...म्हणून सान child सुचित्राची ड्यू डेट आहेत खास
1,entertainment,...श्रावण असाधारणिया मिर्झाची ड्यू डेमॅक्स आहे...
2,entertainment,Ronaldoideo:- दीपिका-रणवीराच्या लग्नाचा एप्रिल...
3,entertainment,स्टीवनideo : दीपिका–मल्हारवीरच्या लग्नाचा जूनन...
4,sports,"सचिनच्या मेलबोर्न काव्यवाचनाची शेवट गोड, मुंबई..."


In [16]:
df_train = pd.DataFrame({0: list(df_train[0])[:cutoff] + list(df_train_aug['0'])[:2*cutoff:2], 1: list(df_train[1])[:cutoff] + list(df_train_aug['1'])[:2*cutoff:2]})
df_train.shape

(966, 2)

In [17]:
Counter(list(df_train[0]))

Counter({'entertainment': 272, 'sports': 88, 'state': 606})

In [18]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [19]:
label_cols = [0]

In [20]:
class MarathiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/marathi/tokenizer/marathi_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [21]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/marathi/tokenizer/marathi_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [22]:
# 30,000 is the vocab size that we chose in sentencepiece
marathi_vocab = Vocab(itos)

In [23]:
tokenizer = Tokenizer(tok_func=MarathiTokenizer, lang='mr')

In [24]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [25]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=marathi_vocab)

In [26]:
data_lm.show_batch()

idx,text
0,"▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल ▁x x bo s ▁संतप्त ▁ग्रामस्थ ांनी ▁राष्ट्रवादी ▁आमदार ाच्या ▁दिशे नं ▁भिरकाव ल्या ▁पाण्याने ▁भरलेल्या ▁घागरी ▁x x bo s <unk> <unk> p e c i <unk> al <unk> ▁r e p <unk> or t ▁: ▁लातूर मध्ये ▁का ▁करावा ▁लागतो य ▁विद्यार्थ्यांना ▁एसटी ने ▁लटक ून ▁प्रवास ? ▁x x bo s ▁यश"
1,"ंच्या ▁विरोधात ▁अ ब्रु ▁नुकसान ी चा ▁दावा ▁x x bo s ▁हा ॅल ि वूड ▁बा ॅल ि वूड ▁ <unk> ▁14 ▁जानेवारी ▁17 ▁) ▁x x bo s <unk> <unk> ▁g am es ▁2018 : ▁डबल ▁ध मा का , ▁जॉन सन ▁आणि ▁महिला ▁टीम ने ▁पटकावले ▁दोन ▁सुवर्णपदक ▁x x bo s ▁औरंगाबाद मध्ये ▁विद्यार्थ्यांना ▁ओलांड ावा ▁लागतो ▁‘ मृत्यू ’ चा ▁ट्रॅक ▁x x bo s"
2,"▁ <unk> ▁ba by m ir za m al ik ▁: ▁सान िया - शो ए ब च्या ▁घरी ▁आला ▁नवा ▁पा हु णा ▁x x bo s <unk> rth day <unk> c ial ▁: ▁... म्हणून ▁संजय ▁दत्त ला ▁त्रि शाला चा ▁ताबा ▁मिळाला ▁नव्हता ▁x x bo s ▁आंबे न ळी ▁बस ▁अपघात ापूर्वी ▁दोन वेळा ▁ड्रायव्हर ▁बदलले , ▁दापोली ▁विद्यापीठाचा ▁अहवाल ▁x x bo s ▁कपिल ▁शर्मा"
3,▁का ▁केलं ▁भट्ट ▁कुटुंब ाचं ▁कौतुक ? ▁x x bo s ▁महा वि त रण चा ▁ग्राहकांना ▁दर वाढ ी चा ▁श ॉक ▁x x bo s ▁दसरा - दिवाळी ▁जवळ ▁आल्याने ▁सराफ ा ▁बाजारात ▁ते जी ▁x x bo s ▁ ...' यांचा ' ▁होणार ▁टीम ▁फडणवीस मध्ये ▁सहभाग ▁x x bo s ▁डे ंग ्यू ▁मलेरिया च्या ▁रुग्ण ांना ▁पपई ▁खाण्या चा ▁डॉक्टर ांचा ▁सल्ला ▁x x bo
4,"▁x x bo s ▁स्टोअर ▁कि पर ▁ते ▁महा ठ ग ▁! ▁x x bo s ▁सहारा ला ▁झटका , ▁महसूल ▁विभागाने ▁' अ ॅ म्ब ी ▁व्हॅली ' ला ▁ठोक लं ▁टाळ ं ▁x x bo s ▁सदा भाऊ ▁खोत ▁भाजप च्या ▁वाटेवर ? ▁x x bo s ▁मंगला ▁एक्स्प्रेस मध्ये ▁जन्म ाली ▁' मंगल ा ' ▁x x bo s ▁बोल ्ड ▁फोटो ▁शेअर ▁केल्यामुळे ▁मान्यता ▁पुन्हा ▁खा"


In [27]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [28]:
# Loading the pretrained language model on kannada wikipedia
learn.load('../../../models/marathi/lm/ULMFiT/fourth_mr_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (966 items)
x: LMTextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (1210 items)
x: LMTextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: LMLabelList
,,,,
Path: .;

Test: LabelList (1210 it

In [29]:
learn.freeze()

In [30]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.763832,6.078973,0.244018,00:00


In [31]:
learn.unfreeze()

In [32]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.318235,5.947213,0.248482,00:00
1,6.155632,5.613878,0.261652,00:00
2,5.952824,5.402398,0.276027,00:00
3,5.773826,5.315521,0.287455,00:00
4,5.640232,5.296645,0.290089,00:00


In [33]:
learn.predict('सीमा भागातील मराठी',n_words=10)

'सीमा भागातील मराठी तून ▁" iv ने तून ▁र ँग ियो ▁गणेश ध'

In [34]:
learn.save_encoder('fine_tuned_enc')

In [35]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=marathi_vocab, label_cols=label_cols, bs=64)

In [36]:
data_clas.show_batch()

text,target
▁x x bo s <unk> ▁l o k ▁ <unk> sa b <unk> ha <unk> ▁e <unk> <unk> e ▁cinema ▁बसलेल i l n s ▁22 19 <unk> <unk> <unk> e : ▁बारामती ▁मतदारसंघ ामध्ये ▁कधी ▁होणार ▁वस्त्रोद्योग ?,state
▁x x bo s <unk> ▁l o k ▁ <unk> sa b <unk> ha <unk> ▁e l e c t i o n s ▁20 19 <unk> ▁d <unk> at e : ▁बारामती ▁मतदारसंघ ामध्ये ▁कधी ▁होणार ▁मतदान ?,state
▁x x bo s <unk> ▁v <unk> id e o <unk> ▁j <unk> ac k i e <unk> <unk> h <unk> ro o f ▁: ▁... ▁म्हणून ▁जॅक ी ▁श्रॉफ ▁लखनऊ त ▁झाला ▁ ट्रॅ फ िक ▁हवालदार,entertainment
▁x x bo s <unk> ▁v <unk> id e o ▁: ▁एक ता ▁कपूर च्या ▁' ग ंदी ▁ब ात ▁2 ' च्या ▁ट्रे लर मध्ये ▁से क्स ▁आणि ▁बोल ्ड ▁सी न्स चा ▁भ डी मार,entertainment
▁x x bo s <unk> ▁v <unk> id e o <unk> ▁j ▁/ ▁41 k <unk> e <unk> <unk> h <unk> o f ▁: ▁... ▁म्हणून ▁जॅक ी ▁नदाफ ▁पाटणादेवी तच ▁झाला ▁ हॉगवॉर्ट् ल्लाह िक ▁हवालदार,entertainment


In [37]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [38]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [39]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (966 items)
x: TextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: CategoryList
entertainment,entertainment,sports,state,sports
Path: .;

Valid: LabelList (1210 items)
x: TextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: CategoryList
s

In [40]:
learn.freeze()

In [41]:
learn.loss_func.func

CrossEntropyLoss()

In [42]:
mcc = MatthewsCorreff()

In [43]:
learn.metrics = [mcc, accuracy]

In [44]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.769102,0.686624,0.515249,0.763636,00:01


In [45]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.489371,0.593895,0.454325,0.736364,00:01


In [46]:
learn.save('second-full')

In [47]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.277432,0.477453,0.611652,0.809091,00:01
1,0.219212,0.424831,0.693245,0.843802,00:01
2,0.169349,0.413545,0.704296,0.850413,00:01
3,0.131351,0.414613,0.700361,0.847107,00:01
4,0.105826,0.421067,0.695630,0.844628,00:01


Better model found at epoch 0 with accuracy value: 0.8090909123420715.
Better model found at epoch 1 with accuracy value: 0.8438016772270203.
Better model found at epoch 2 with accuracy value: 0.8504132032394409.


In [48]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (966 items)
x: TextList
▁x x bo s ▁... म्हणून ▁सान िया ▁मिर्झा ची ▁ड्यू ▁डे ट ▁आहे ▁खास,▁x x bo s <unk> ▁v ide o ▁: ▁दीपिका - रण वीर च्या ▁लग्न ाचा ▁मे न्यू ▁झाला ▁ली क !,▁x x bo s ▁सचिन च्या ▁रणजी ▁कारकीर्दी चा ▁शेवट ▁गोड , ▁मुंबई चा ▁विजय,▁x x bo s ▁पुरंदरे ंना ▁महाराष्ट्र ▁भूषण ▁पुरस्कार ाविरोधात ▁संभाजी ▁ब्रिगेड ▁रस्त्यावर,▁x x bo s ▁जयपूर ▁पोलिस ांच्या ▁जाहिराती वर ▁बु म राह ▁संताप ला , ▁सु ना वले ▁खडे बोल
y: CategoryList
entertainment,entertainment,sports,state,sports
Path: .;

Valid: LabelList (1210 items)
x: TextList
▁x x bo s <unk> ▁c w g ▁2018 ▁: ▁सं जित ा ▁ चा नू ची ▁सुवर्ण भर ारी , ▁भारताच्या ▁खात्यात ▁दुसरे ▁सुवर्ण ▁!,▁x x bo s ▁कर्मचार् ▁यांच्या ▁हल ग र्ज ी पणामुळे ▁होणार ▁होते ▁जिवंत ▁बाळ ावर ▁अंत्यसंस्कार,▁x x bo s ▁श्रीलंकेचा ▁भारतावर ▁7 ▁गडी ▁राखून ▁विजय,▁x x bo s ▁औरंगाबाद ▁महापौर पदाचा ▁ति ढा ▁अखेर ▁सुट ला,▁x x bo s ▁औरंगाबाद ▁विद्यापीठाचा ▁भो ंग ळ ▁कारभार ; ▁गोदाम ात ▁दिले ▁विद्यार्थ्यांनी ▁पेपर
y: CategoryList
s

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,sports,state,entertainment
0,काजोल पुन्हा माझ्या आयुष्यात येणार नाही - करण ...,entertainment,entertainment,0.0221199,0.00575353,0.972126
1,विराटला चीअर करण्यासाठी अनुष्का सिडनीत,entertainment,entertainment,0.151038,0.00379612,0.845165
2,संतोषचा अड्डा!,entertainment,entertainment,0.0461972,0.0110578,0.942745
3,मुरूड समुद्रात बुडालेल्या विद्यार्थ्याचा मृतदे...,state,state,0.0176764,0.976678,0.00564613
4,"गोव्यात शिवसेना वेलिंगकरांसोबत,लवकरच युतीची घोषणा",state,state,0.024485,0.968073,0.00744244


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.8454545454545455

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.6911086300041109

In [52]:
df_result.to_csv('inltk_headlines_mr_with_aug.csv', index=False)